In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import pickle

with open('../input/pretrained-for-riiid/user_ele_dict.pkl', 'rb') as f:
    user_ele_dict = pickle.load(f)

with open('../input/pretrained-for-riiid/user_num_dict.pkl', 'rb') as f:
    user_num_dict = pickle.load(f)

In [ ]:
import copy
minus_ele_dict = copy.deepcopy(user_ele_dict)
minus_num_dict = copy.deepcopy(user_num_dict)

In [ ]:
print(minus_num_dict[115])

In [ ]:
question_dtype = {
    'question_id':'int16',
    'tags':'object'
}
questions_data = pd.read_csv('../input/riiid-test-answer-prediction/questions.csv',
                             usecols = question_dtype.keys(), 
                             dtype = question_dtype)

questions_data.tags.fillna('92',inplace=True)

def gen_vec(row):
    row['vec'] = np.zeros(188)
    index_list = row.tags.split()
    for index_ in index_list:
        row.vec[int(index_)] = 1.0
    return row

questions_data = questions_data.apply(gen_vec, axis='columns')

In [ ]:
questions_data

In [ ]:
data = pd.read_pickle("../input/riiid-train-data-multiple-formats/riiid_train.pkl.gzip")
data['prior_question_had_explanation'] = data['prior_question_had_explanation'].astype('bool')
data = data[data.content_type_id == 0]
data.head()

In [ ]:
len(data)

In [ ]:
users_i_counts = data.user_id.value_counts()
user_list = users_i_counts.keys()
print(user_list[:5])
print(users_i_counts.head())

In [ ]:
len(user_list)

In [ ]:
sum(users_i_counts.values>10)

In [ ]:
users_i_counts.values[390685]

In [ ]:
data[data.user_id ==115][-1:]#.answered_correctly.values[0]

In [ ]:
def minus_one(user_list_, ele_dict=minus_ele_dict, num_dict=minus_num_dict, q_data=questions_data,data=data):
    for user in tqdm(user_list_):
        user_data = data[data.user_id==user][-1:]
        num_dict[user] = num_dict[user] - q_data.vec[user_data.content_id.values[0]]
        ele_dict[user] = ele_dict[user] - q_data.vec[user_data.content_id.values[0]]*user_data.answered_correctly.values[0]

In [ ]:
minus_one(user_list)

In [ ]:
print(user_num_dict[455973631]-minus_num_dict[455973631])
print(user_ele_dict[455973631]-minus_ele_dict[455973631])

In [ ]:
data[data.user_id==455973631][-1:]

In [ ]:
questions_data[questions_data.question_id==10708]